In [4]:
# import wave
# import torch
import time
import threading
from TTS.api import TTS
# import requests
# import sounddevice as sd
# import json
# from IPython.display import Audio
# from playsound import playsound
import pygame
# import numpy as np
# import os
import queue
# from pydub import AudioSegment
# import os

In [20]:
TTS.list_models()

['tts_models/multilingual/multi-dataset/your_tts',
 'tts_models/bg/cv/vits',
 'tts_models/cs/cv/vits',
 'tts_models/da/cv/vits',
 'tts_models/et/cv/vits',
 'tts_models/ga/cv/vits',
 'tts_models/en/ek1/tacotron2',
 'tts_models/en/ljspeech/tacotron2-DDC',
 'tts_models/en/ljspeech/tacotron2-DDC_ph',
 'tts_models/en/ljspeech/glow-tts',
 'tts_models/en/ljspeech/speedy-speech',
 'tts_models/en/ljspeech/tacotron2-DCA',
 'tts_models/en/ljspeech/vits',
 'tts_models/en/ljspeech/vits--neon',
 'tts_models/en/ljspeech/fast_pitch',
 'tts_models/en/ljspeech/overflow',
 'tts_models/en/ljspeech/neural_hmm',
 'tts_models/en/vctk/vits',
 'tts_models/en/vctk/fast_pitch',
 'tts_models/en/sam/tacotron-DDC',
 'tts_models/en/blizzard2013/capacitron-t2-c50',
 'tts_models/en/blizzard2013/capacitron-t2-c150_v2',
 'tts_models/es/mai/tacotron2-DDC',
 'tts_models/es/css10/vits',
 'tts_models/fr/mai/tacotron2-DDC',
 'tts_models/fr/css10/vits',
 'tts_models/uk/mai/glow-tts',
 'tts_models/uk/mai/vits',
 'tts_models/zh

In [21]:
#Coqui TTS
# tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=False)

tts = TTS(model_name='tts_models/en/ek1/tacotron2', progress_bar=True, gpu=False)

 > Downloading model to /Users/anshulwadhawan/Library/Application Support/tts/tts_models--en--ek1--tacotron2


100%|██████████| 347M/347M [06:39<00:00, 869kiB/s]  


 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /Users/anshulwadhawan/Library/Application Support/tts/vocoder_models--en--ek1--wavegrad


  1%|          | 2.21M/190M [00:01<01:15, 2.50MiB/s]

In [3]:
#Eleven Labs
url = "https://api.elevenlabs.io/v1/text-to-speech/TxGEqnHWrfWFTfGW9XjX"
headers = {
    "xi-api-key": "ff79eb4ad43bb12b04a3968a5b92125b"
}

In [11]:
text ="A muffin is an individually portioned baked product, however the term can refer to one of two distinct items: a part-raised flatbread that is baked and then cooked on a griddle, or a quickbread that is chemically leavened and then baked in a mold."
text_small = "Muffin is a baked item, either a flatbread cooked on griddle or a quickbread leavened chemically and baked in a mold."
text_hi = "Hi"

In [16]:
q = queue.Queue()
done = False
# pygame.mixer.init()

def streaming_write(sentence):
    for word in sentence.split():
        with open("streaming_input.txt", "a") as file:
            file.write(word+" ")
        time.sleep(0.2)

def convert_to_audio(chunk, name):
    # samples = tts.synthesize(chunk, speaker)
    # with wave.open("outputs/example"+ str(name) +".wav", "w") as fp:
    #     fp.setparams((1, 2, 24000, len(samples), "NONE", "NONE"))
    #     fp.writeframes(samples)

    filepath = "outputs/example"+ str(name) +".wav"
    start = time.time()
    # tts.tts_to_file(chunk, speaker=tts.speakers[0], language="en", file_path=filepath)
    tts.tts_to_file(chunk, file_path=filepath)

    # data = {
    #     "text": chunk,
    #     "voice_settings": {
    #         "stability": 0,
    #         "similarity_boost": 0
    #     }
    # }

    # response = requests.post(url, data=json.dumps(data), headers=headers)

    # if response.status_code == 200:
    #     with open(filepath, 'wb') as f:
    #         f.write(response.content)
    # else:
    #     print("Error")

    # pygame.mixer.music.load(filepath)
    # pygame.mixer.music.play()
    # while pygame.mixer.music.get_busy():
    #     # pygame.time.Clock().tick(10)
    #     continue

    end = time.time()
    print(end-start)
    q.put(filepath)
    
def streamingTTS():
    content=""
    while True:
        with open("streaming_input.txt", "r") as file:
            new_content = file.read()
            print(len(content), len(new_content))
            if len(new_content) > len(content):
                convert_to_audio(new_content[len(content):], len(new_content))
                content = new_content
            else:
                done = True
                break
        # time.sleep(0.3)

def audioplayer():
    pygame.mixer.init()
    while not done:
        while not q.empty():
            filepath = q.get()
            pygame.mixer.music.load(filepath)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                continue


In [17]:
write_thread = threading.Thread(target=streaming_write, args = (text,) )
write_thread.start()

# write_thread = threading.Thread(target=streaming_write, args = (text_small,) )
# write_thread.start()

In [18]:
player_thread = threading.Thread(target=audioplayer)
player_thread.start()

In [19]:
tts_thread = threading.Thread(target=streamingTTS)
tts_thread.start()

0 38
 > Text splitted to sentences.
['A muffin is an individually portioned']


ə mʌfən ɪz ən ɪndɪvɪd͡ʒuəli pɔɹʃənd
 [!] Character '͡' not found in the vocabulary. Discarding it.


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


 > Processing time: 12.655647993087769
 > Real-time factor: 4.30749935551348
12.706302165985107
38 248
 > Text splitted to sentences.
['baked product, however the term can refer to one of two distinct items: a part-raised flatbread that is baked and then cooked on a griddle, or a quickbread that is chemically leavened and then baked in a mold.']
 > Processing time: 10.236576080322266
 > Real-time factor: 0.7767899020259965
10.311808824539185
248 248
